In [ ]:
Nf.ready(function () {



    var page 

    var cont = 0

    var mensagem 

    const hideContainerNotify = () => {

        if ($(".container-notifies").children().length === 0) {

            if ($(".container-notifies").children().length === 0) {



            hasData = false;



        }



    }

    const getDateFormat = (month, day, hour, min) => {

        month += 1;

        var monthString = "";



        switch (parseInt(month)) {

            case 1:

                monthString = "Jan";

                break;

            case 2:

                monthString = "Fev";

                break;

            case 3:

                monthString = "Mar";

                break;

            case 4:

                monthString = "Apr";

                break;

            case 5:

                monthString = "May";

                break;

            case 6:

                monthString = "Jun";

                break;

            case 7:

                monthString = "Jul";

                break;

            case 8:

                monthString = "Ago";

                break;

            case 9:

                monthString = "Set";

                break;

            case 10:

                monthString = "Out";

                break;

            case 11:

                monthString = "Nov";

                break;

            case 12:

                monthString = "Dez";

                break;

            default:

                monthString = "UNDEFINED";

                break;



        }



        if (parseInt(hour) < 10) {

            hour = "0" + hour;



        }



        if (parseInt(min) < 10) {

            min = "0" + min;



        }



        return (monthString + ". " + month + ", " + day + " " + hour + ":" + min);



    }



    const changeSpanValue = () => {

        $("#qtd-notifies").text($(".container-notifies").children().length);



    }



    const createNotificationIcon = () => {

        const not = "<div>" +

                        "<div id='notifies' :class='classIcon' v-on:click='handleClickNotIcon'>" +

                        "</div>" +

                        "<div id='qtd-notifies' :class=''>" +

                        "</div>" +

                        "<div class='container-notifies'>" +

                            "<div class='border-bottom' v-for='(notification, index) in notifications' :key='index' :id='notification.id'>" +

                                "<div :title='notification.message' class='bi bi-info-circle-fill'></div>" +

                                "<div id='container-title-main'>" +

                                    "<div id='container-title'>" +

                                        "<h4 id='title-notify' v-on:click='handleClickRedirect'> {{message}}</h4>" +

                                        "<p>{{ notification.change_time }}</p>" +

                                    "</div>" +

                                "</div>" +

                                "<div id='btn-cancel' :name='notification.index' class='btn-check bi bi-x-circle-fill cancel' v-on:click='updateVisibility'></div>" +

                            "</div>" +

                            "<div v-if='no_data'>" +

                                "<h3 id='not-data'>You don't have unread notifications.</h3>" +

                            "</div>" +

                        "</div>" +

                    "</div>"

        return new Vue({

            el: "#notification-panel",

            template: not,

            data (

                

            ) {

                return {

                    classIcon: "bi-bell-slash icon",

                    message: mensagem,

                    notifications: [],

                    notifications_to_update: [],

                    no_data: true



                }



            },

            mounted() {

                setInterval(() => {

                    MessageProcessor.process({

                        serviceId: "app.service.BRPC_DEVELOPMENT.brpc_dev_model_notification_getList_customized",

                        data: {

                            "start": "0",

                            "limit": "1000",

                            "visible": "1"



                        },

                        success: (ev) => {

                            

                            if(cont<ev.results.length )

                            {

                                if(ev.results[cont].approver_business_unit)

                                {

                                    mensagem = 'New Service Request'

                                    page = "/spl2/BRPC_DEVELOPMENT/brpc_dev_model_new_services_request_create_page"

                                   

                                }

                                else{

                                    page = "/spl2/BRPC_DEVELOPMENT/brpc_dev_model_extra_services_list_page"

                                    mensagem = 'New Extra Service'

                                   

                                }

                                cont+=1

                            }

                    

                            if (ev.results.length === 0) {

                                $("#notifies").removeClass("bi-bell");

                                $("#notifies").addClass("bi-bell-slash");

                                this.no_data = true;



                            } else {

                                $("#notifies").addClass("bi-bell");

                                $("#notifies").removeClass("bi-bell-slash");

                                this.no_data = false;



                            }



                            $("#qtd-notifies").text(ev.results.length);

                            this.notifications = this.handleNotifies(ev.results);




                        },



                    });

                }, 2000);



            },

            beforeCreate() {

                MessageProcessor.process({

                    serviceId: "app.service.BRPC_DEVEELOPMENT.brpc_dev_model_notification_getList_customized",

                    data: {

                        "start": "0",

                        "limit": "1000",

                        "visible": "1"



                    },

                    success: (ev) => {

                        if (ev.results.length === 0) {

                            $("#notifies").removeClass("bi-bell");

                            $("#notifies").addClass("bi-bell-slash");

                            this.no_data = true;



                        } else {

                            $("#notifies").addClass("bi-bell");

                            $("#notifies").removeClass("bi-bell-slash");

                            this.no_data = false;



                        }



                        $("#qtd-notifies").text(ev.results.length);

                        this.notifications = this.handleNotifies(ev.results);



                    },



                });



            },

            methods: {

                handleNotifies (data) {

                    this.notifications_to_update = data;



                    var list = [];



                    data.forEach((not, index) => {

                        var timestamp = new Date(parseInt(not.change_time));

                        var date = getDateFormat(timestamp.getMonth(), timestamp.getDate(), timestamp.getHours(), timestamp.getMinutes());

                        not.change_time = date;

                        not.index = index;



                        list.push(not);



                    });



                    return list;



                },



                handleClickNotIcon () {

                    $(".container-notifies").stop().slideToggle();



                },



                updateVisibility (ev) {

                    var req = this.notifications_to_update[parseInt($(ev.target).attr("name"))];

                    req.visible = 0;



                    MessageProcessor.process({

                        serviceId: "app.service.BRPC_DEVELOPMENT.brpc_dev_model_notification_update",

                        data: req,

                        success: (ev) => {

                            console.log(ev);



                        }



                    });



                    $(ev.target).parent().fadeOut(1000);



                    setTimeout(() => {

                        $(ev.target).parent().remove();

                        hideContainerNotify();

                        changeSpanValue();



                    }, 1001);



                },

                

                handleClickRedirect (ev) {

                   

                    U.PageOpenMgt.open("_blank", page);

                    

                }



           }



        });



    }



    



    setTimeout(() => {

        createNotificationIcon();

    }, 300);



    $("#redirect_button_extra").hide();



});